# Naive Bayes

*Esse notebook foi gerado a partir de exercícios da disciplina de Linguagem de Programação Aplicada do curso de Pós Graduação em Inteligência Artifical Aplicada da UFPR.*

Discente: Tiago José da Silva

Professor: Dr. Alexander Kutzke 

## Antes de começarmos

Esse notebook expõe um classificador de spam utilizando uma implementação de Naive Bayes.

Para testarmos a implementação usaremos um conjunto de e-mails que está disponível em:
    
Ao fazer download dos arquivos, extraio-os para um diretório fazendo com que cada um dos dos arquivos se transforme num subdiretório.

Após isso, modifique a variável path de modo a refletir a estrutura criada.

In [183]:
path = 'naive_bayes/emails/*/*'

## O Código

In [151]:
import math
import random
import re
from collections import defaultdict, Counter
from typing import TypeVar, List, Tuple, Dict, Iterable, NamedTuple, Set

X = TypeVar('X')  # generic type to represent a data point

- Importei a lib nltk.stem para obter os radicais das palavras.

Mais informações sobre a lib: https://www.nltk.org/howto/stem.html

In [152]:
from nltk.stem import PorterStemmer
stemmer = PorterStemmer()

In [153]:
def split_data(data: List[X], prob: float) -> Tuple[List[X], List[X]]:
    """Split data into fractions [prob, 1 - prob]"""
    data = data[:]  # Make a shallow copy
    random.shuffle(data)  # because shuffle modifies the list.
    cut = int(len(data) * prob)  # Use prob to find a cutoff
    return data[:cut], data[cut:]  # and split the shuffled list there.

In [184]:
def tokenize(text: str, stem: bool = False, verify_number: bool = False) -> Set[str]:
    text = text.lower()  # Convert to lowercase,
    all_words = re.findall("[a-z0-9']+", text)  # extract the words, and
    if verify_number:
        all_words.append("contain-numbers:" + str(len(re.findall('\d+', text)) > 0))
    if stem:
        all_words = [stemmer.stem(w) for w in all_words]
    return set(all_words)  # remove duplicates.

    assert tokenize("Data Science is science") == {"data", "science", "is"}

In [155]:
class Message(NamedTuple):
    text: str
    is_spam: bool

- Adicionei três novos parâmetros no construtor do classificador NaiveBayes.
    - min_count: Permite definir a quantidade mínima de vezes que um token precisa aparecer para ser considerado no classificador;
    - stem: Caso True, considerará apenas os radicais das palavras na hora da geração dos tokens;
    - verify_number: Caso True, um token chamado *contain-numbers:True* ou *contain-numbers:False* será adicionado para os cenários em que existir e que não existir a presença de números no texto, respectivamente.

In [156]:
class NaiveBayesClassifier:
    def __init__(self, k: float = 0.5,
                 min_count: int = 1,
                 stem: bool = False,
                 verify_number: bool = False) -> None:

        self.k = k  # smoothing factor
        self.min_count = min_count
        self.stem = stem
        self.verify_number = verify_number

        self.tokens: Set[str] = set()
        self.tokens_count: Dict[str, int] = defaultdict(int)
        self.token_spam_counts: Dict[str, int] = defaultdict(int)
        self.token_ham_counts: Dict[str, int] = defaultdict(int)
        self.spam_messages = self.ham_messages = 0

    def train(self, messages: Iterable[Message]) -> None:
        for message in messages:
            # Increment message counts
            if message.is_spam:
                self.spam_messages += 1
            else:
                self.ham_messages += 1

            # Increment word counts
            for token in tokenize(message.text, stem=self.stem, verify_number=self.verify_number):
                self.tokens.add(token)
                self.tokens_count[token] += 1
                if message.is_spam:
                    self.token_spam_counts[token] += 1
                else:
                    self.token_ham_counts[token] += 1

    def probabilities(self, token: str) -> Tuple[float, float]:
        """returns P(token | spam) and P(token | not spam)"""
        spam = self.token_spam_counts[token]
        ham = self.token_ham_counts[token]

        p_token_spam = (spam + self.k) / (self.spam_messages + 2 * self.k)
        p_token_ham = (ham + self.k) / (self.ham_messages + 2 * self.k)

        return p_token_spam, p_token_ham

    def predict(self, text: str) -> float:
        text_tokens = tokenize(text, stem=self.stem, verify_number=self.verify_number)
        log_prob_if_spam = log_prob_if_ham = 0.0

        # Iterate through each word in our vocabulary.
        for token in self.tokens:
            if self.tokens_count[token] < self.min_count:
                continue

            prob_if_spam, prob_if_ham = self.probabilities(token)

            # If *token* appears in the message,
            # add the log probability of seeing it;
            if token in text_tokens:
                log_prob_if_spam += math.log(prob_if_spam)
                log_prob_if_ham += math.log(prob_if_ham)

            # otherwise add the log probability of _not_ seeing it
            # which is log(1 - probability of seeing it)
            else:
                log_prob_if_spam += math.log(1.0 - prob_if_spam)
                log_prob_if_ham += math.log(1.0 - prob_if_ham)

        prob_if_spam = math.exp(log_prob_if_spam)
        prob_if_ham = math.exp(log_prob_if_ham)

        return prob_if_spam / (prob_if_spam + prob_if_ham)

## Testes de modelo

### Teste Original

In [157]:
messages = [Message("spam rules", is_spam=True),
            Message("ham rules", is_spam=False),
            Message("hello ham", is_spam=False)]

model = NaiveBayesClassifier(k=0.5)
model.train(messages)

assert model.tokens == {"spam", "ham", "rules", "hello"}
assert model.tokens_count == {"spam": 1, "rules": 2, "ham": 2, "hello": 1}
assert model.spam_messages == 1
assert model.ham_messages == 2
assert model.token_spam_counts == {"spam": 1, "rules": 1}
assert model.token_ham_counts == {"ham": 2, "rules": 1, "hello": 1}

text = "hello spam"

probs_if_spam = [
    (1 + 0.5) / (1 + 2 * 0.5),      # "spam"  (present)
    1 - (0 + 0.5) / (1 + 2 * 0.5),  # "ham"   (not present)
    1 - (1 + 0.5) / (1 + 2 * 0.5),  # "rules" (not present)
    (0 + 0.5) / (1 + 2 * 0.5)       # "hello" (present)
]

probs_if_ham = [
    (0 + 0.5) / (2 + 2 * 0.5),      # "spam"  (present)
    1 - (2 + 0.5) / (2 + 2 * 0.5),  # "ham"   (not present)
    1 - (1 + 0.5) / (2 + 2 * 0.5),  # "rules" (not present)
    (1 + 0.5) / (2 + 2 * 0.5),      # "hello" (present)
]

p_if_spam = math.exp(sum(math.log(p) for p in probs_if_spam))
p_if_ham = math.exp(sum(math.log(p) for p in probs_if_ham))

# Should be about 0.83
assert math.isclose(model.predict(text), p_if_spam / (p_if_spam + p_if_ham))

### Teste com quantidade mínima de tokens = 2

In [158]:
messages = [Message("spam rules", is_spam=True),
            Message("ham rules", is_spam=False),
            Message("hello ham", is_spam=False)]

model = NaiveBayesClassifier(k=0.5, min_count = 2)
model.train(messages)

assert model.tokens == {"spam", "ham", "rules", "hello"}
assert model.tokens_count == {"spam": 1, "rules": 2, "ham": 2, "hello": 1}
assert model.spam_messages == 1
assert model.ham_messages == 2
assert model.token_spam_counts == {"spam": 1, "rules": 1}
assert model.token_ham_counts == {"ham": 2, "rules": 1, "hello": 1}

text = "hello spam"

probs_if_spam = [
    1 - (0 + 0.5) / (1 + 2 * 0.5),  # "ham"   (not present)
    1 - (1 + 0.5) / (1 + 2 * 0.5),  # "rules" (not present)
]

probs_if_ham = [
    1 - (2 + 0.5) / (2 + 2 * 0.5),  # "ham"   (not present)
    1 - (1 + 0.5) / (2 + 2 * 0.5),  # "rules" (not present)
]

p_if_spam = math.exp(sum(math.log(p) for p in probs_if_spam))
p_if_ham = math.exp(sum(math.log(p) for p in probs_if_ham))

# Should be about 0.69
assert math.isclose(model.predict(text), p_if_spam / (p_if_spam + p_if_ham))

### Teste com verify_number = True

In [159]:
messages = [Message("spam rules 10", is_spam=True),
            Message("ham rules", is_spam=False),
            Message("hello ham", is_spam=False)]

model = NaiveBayesClassifier(k=0.5, verify_number = True)
model.train(messages)

assert model.tokens == {"spam", "ham", "rules", "hello", "10", "contain-numbers:True", "contain-numbers:False"}
assert model.tokens_count == {"spam": 1, "rules": 2, "ham": 2, "hello": 1, "10": 1, "contain-numbers:True": 1, "contain-numbers:False": 2}
assert model.spam_messages == 1
assert model.ham_messages == 2
assert model.token_spam_counts == {"spam": 1, "rules": 1, "10": 1, "contain-numbers:True": 1}
assert model.token_ham_counts == {"ham": 2, "rules": 1, "hello": 1, "contain-numbers:False": 2}

text = "hello spam"

probs_if_spam = [
    (1 + 0.5) / (1 + 2 * 0.5),      # "spam"  (present)
    1 - (0 + 0.5) / (1 + 2 * 0.5),  # "ham"   (not present)
    1 - (1 + 0.5) / (1 + 2 * 0.5),  # "rules" (not present)
    (0 + 0.5) / (1 + 2 * 0.5),      # "hello" (present)
    1 - (1 + 0.5) / (1 + 2 * 0.5),  # "10" (not present)
    1 - (1 + 0.5) / (1 + 2 * 0.5),  # "contain-numbers:True" (not present)
    (0 + 0.5) / (1 + 2 * 0.5),      # "contain-numbers:False" (present)
]

probs_if_ham = [
    (0 + 0.5) / (2 + 2 * 0.5),      # "spam"  (present)
    1 - (2 + 0.5) / (2 + 2 * 0.5),  # "ham"   (not present)
    1 - (1 + 0.5) / (2 + 2 * 0.5),  # "rules" (not present)
    (1 + 0.5) / (2 + 2 * 0.5),      # "hello" (present)
    1 - (0 + 0.5) / (2 + 2 * 0.5),  # "10"   (not present)
    1 - (0 + 0.5) / (2 + 2 * 0.5),  # "contain-numbers:True"   (not present)
    (2 + 0.5) / (2 + 2 * 0.5)       # "contain-numbers:False" (present)
]

p_if_spam = math.exp(sum(math.log(p) for p in probs_if_spam))
p_if_ham = math.exp(sum(math.log(p) for p in probs_if_ham))

# Should be about 0.12
assert math.isclose(model.predict(text), p_if_spam / (p_if_spam + p_if_ham))

### Teste com stem = True

In [160]:
messages = [Message("spam working", is_spam=True),
            Message("hard job", is_spam=False),
            Message("jobs", is_spam=False)]

model = NaiveBayesClassifier(k=0.5, stem = True)
model.train(messages)

assert model.tokens == {"spam", "work", "hard", "job"}
assert model.tokens_count == {"spam": 1, "work": 1, "hard": 1, "job": 2}
assert model.spam_messages == 1
assert model.ham_messages == 2
assert model.token_spam_counts == {"spam": 1, "work": 1}
assert model.token_ham_counts == {"hard": 1, "job": 2}

text = "job work"

probs_if_spam = [
    1 - (1 + 0.5) / (1 + 2 * 0.5),  # "spam"  (not present)
    (1 + 0.5) / (1 + 2 * 0.5),      # "work"   (present)
    1 - (0 + 0.5) / (1 + 2 * 0.5),  # "hard" (not present)
    (0 + 0.5) / (1 + 2 * 0.5)       # "job" (present)
]

probs_if_ham = [
    1 - (0 + 0.5) / (2 + 2 * 0.5),  # "spam"  (not present)
    (0 + 0.5) / (2 + 2 * 0.5),      # "work"   (present)
    1 - (1 + 0.5) / (2 + 2 * 0.5),  # "hard" (not present)
    (2 + 0.5) / (2 + 2 * 0.5),      # "job" (present)
]

p_if_spam = math.exp(sum(math.log(p) for p in probs_if_spam))
p_if_ham = math.exp(sum(math.log(p) for p in probs_if_ham))

# Should be about 0.38
assert math.isclose(model.predict(text), p_if_spam / (p_if_spam + p_if_ham))

## Testes massivos

In [161]:
def execute(model):
    import glob

    data: List[Message] = []

    # glob.glob returns every filename that matches the wildcarded path
    for filename in glob.glob(path):
        is_spam = "ham" not in filename

        # There are some garbage characters in the emails, the errors='ignore'
        # skips them instead of raising an exception.
        with open(filename, errors='ignore') as email_file:
            # s = ' '.join(email_file)
            # data.append(Message(s, is_spam))
            for line in email_file:
                if line.startswith("Subject:"):
                    subject = line.lstrip("Subject: ")
                    data.append(Message(subject, is_spam))
                    break  # done with this file

    random.seed(0)  # just so you get the same answers as me
    train_messages, test_messages = split_data(data, 0.75)

    # model = NaiveBayesClassifier()
    model.train(train_messages)

    predictions = [(message, model.predict(message.text))
                   for message in test_messages]

    # Assume that spam_probability > 0.5 corresponds to spam prediction
    # and count the combinations of (actual is_spam, predicted is_spam)
    confusion_matrix = Counter((message.is_spam, spam_probability > 0.5)
                               for message, spam_probability in predictions)


    def p_spam_given_token(token: str, model: NaiveBayesClassifier) -> float:
        prob_if_spam, prob_if_ham = model.probabilities(token)

        return prob_if_spam / (prob_if_spam + prob_if_ham)

    words = sorted(model.tokens, key=lambda t: p_spam_given_token(t, model))

    print("spammiest_words", words[-10:])
    print("hammiest_words", words[:10])
    
    return confusion_matrix

### Teste sem nenhum parâmetro (solução original)

In [162]:
original_result = execute(NaiveBayesClassifier())

spammiest_words ['per', 'guaranteed', 'account', 'sale', 'attn', 'zzzz', 'systemworks', 'money', 'rates', 'adv']
hammiest_words ['spambayes', '2', 'users', 'razor', 'zzzzteana', 'sadev', 'ouch', 'apt', 'bliss', 'wedded']


In [163]:
original_result

Counter({(False, False): 668,
         (True, True): 85,
         (True, False): 54,
         (False, True): 18})

### Testes com parâmetro min_count

#### Teste com parâmetro min_count = 2

In [164]:
min_count_2_result = execute(NaiveBayesClassifier(min_count = 2))

spammiest_words ['per', 'guaranteed', 'account', 'sale', 'attn', 'zzzz', 'systemworks', 'money', 'rates', 'adv']
hammiest_words ['spambayes', '2', 'users', 'razor', 'zzzzteana', 'sadev', 'ouch', 'apt', 'bliss', 'wedded']


In [165]:
min_count_2_result

Counter({(False, False): 631,
         (True, True): 106,
         (True, False): 33,
         (False, True): 55})

#### Teste com parâmetro min_count = 3

In [166]:
min_count_3_result = execute(NaiveBayesClassifier(min_count = 3))

spammiest_words ['per', 'guaranteed', 'account', 'sale', 'attn', 'zzzz', 'systemworks', 'money', 'rates', 'adv']
hammiest_words ['spambayes', '2', 'users', 'razor', 'zzzzteana', 'sadev', 'ouch', 'apt', 'bliss', 'wedded']


In [167]:
min_count_3_result

Counter({(False, False): 614,
         (True, True): 111,
         (True, False): 28,
         (False, True): 72})

#### Teste com parâmetro min_count = 4

In [168]:
min_count_4_result = execute(NaiveBayesClassifier(min_count = 4))

spammiest_words ['per', 'guaranteed', 'account', 'sale', 'attn', 'zzzz', 'systemworks', 'money', 'rates', 'adv']
hammiest_words ['spambayes', '2', 'users', 'razor', 'zzzzteana', 'sadev', 'ouch', 'apt', 'bliss', 'wedded']


In [169]:
min_count_4_result

Counter({(False, False): 604,
         (False, True): 82,
         (True, True): 107,
         (True, False): 32})

#### Teste com parâmetro min_count = 5

In [170]:
min_count_5_result = execute(NaiveBayesClassifier(min_count = 5))

spammiest_words ['per', 'guaranteed', 'account', 'sale', 'attn', 'zzzz', 'systemworks', 'money', 'rates', 'adv']
hammiest_words ['spambayes', '2', 'users', 'razor', 'zzzzteana', 'sadev', 'ouch', 'apt', 'bliss', 'wedded']


In [171]:
min_count_5_result

Counter({(False, False): 587,
         (False, True): 99,
         (True, True): 101,
         (True, False): 38})

#### Conclusão sobre o parâmetro min_count

In [172]:
import pandas as pd

df_min_count_test = pd.DataFrame([dict(original_result),
                      dict(min_count_2_result),
                      dict(min_count_3_result),
                      dict(min_count_4_result),
                      dict(min_count_5_result)],
                      index = ['original (min_count = 1)', 
                               'min_count = 2', 
                               'min_count = 3', 
                               'min_count = 4', 
                               'min_count = 5'])

min_count_test_success = df_min_count_test[(False, False)] + df_min_count_test[(True, True)]
min_count_test_fail = df_min_count_test[(False, True)] + df_min_count_test[(True, False)]

df_min_count_test['Total Success']= min_count_test_success
df_min_count_test['Total Fail']= min_count_test_fail
df_min_count_test

,"(False, False)","(True, True)","(True, False)","(False, True)",Total Success,Total Fail
original (min_count = 1),668,85,54,18,753,72
min_count = 2,631,106,33,55,737,88
min_count = 3,614,111,28,72,725,100
min_count = 4,604,107,32,82,711,114
min_count = 5,587,101,38,99,688,137


Ao aumentar o parâmetro min_count notamos uma diminuição crescente de efetividade, o que mostra um **resultado negativo para a classificação de spams**.

### Teste com stem = True

In [173]:
stem_result = execute(NaiveBayesClassifier(stem = True))

spammiest_words ['rep', 'clearanc', 'per', 'assist', 'account', 'attn', 'zzzz', 'systemwork', 'money', 'adv']
hammiest_words ['spambay', '2', 'razor', 'zzzzteana', 'sadev', 'ouch', 'sell', 'apt', 'wed', 'bliss']


In [174]:
stem_result

Counter({(False, False): 656,
         (True, True): 91,
         (True, False): 48,
         (False, True): 30})

In [175]:
import pandas as pd

df_stem_test = pd.DataFrame([dict(original_result),
                             dict(stem_result)],
                      index = ['original (sem stem)', 
                               'com stem'])

stem_test_success = df_stem_test[(False, False)] + df_stem_test[(True, True)]
stem_test_fail = df_stem_test[(False, True)] + df_stem_test[(True, False)]

df_stem_test['Total Success']= stem_test_success
df_stem_test['Total Fail']= stem_test_fail
df_stem_test

,"(False, False)","(True, True)","(True, False)","(False, True)",Total Success,Total Fail
original (sem stem),668,85,54,18,753,72
com stem,656,91,48,30,747,78


#### Conclusão sobre o teste com stem

Há uma ligeira melhora na identificação de spams, porém há também um aumento no número de e-mails não spams identificados como spam, o que não é nada interessante.

No geral o número de sucessos caiu se compararmos com a implementação original.

### Teste com verify_number = True

In [176]:
verify_number_result = execute(NaiveBayesClassifier(verify_number = True))

spammiest_words ['per', 'guaranteed', 'account', 'sale', 'attn', 'zzzz', 'systemworks', 'money', 'rates', 'adv']
hammiest_words ['spambayes', '2', 'users', 'razor', 'zzzzteana', 'sadev', 'ouch', 'apt', 'bliss', 'wedded']


In [177]:
verify_number_result

Counter({(False, False): 666,
         (True, True): 87,
         (True, False): 52,
         (False, True): 20})

In [178]:
import pandas as pd

df_verify_number_test = pd.DataFrame([dict(original_result),
                                      dict(verify_number_result)],
                                      index = ['original (sem verify-number)', 
                                      'com verify_number'])

verify_number_test_success = df_verify_number_test[(False, False)] + df_verify_number_test[(True, True)]
verify_number_test_fail = df_verify_number_test[(False, True)] + df_verify_number_test[(True, False)]

df_verify_number_test['Total Success']= verify_number_test_success
df_verify_number_test['Total Fail']= verify_number_test_fail
df_verify_number_test

,"(False, False)","(True, True)","(True, False)","(False, True)",Total Success,Total Fail
original (sem verify-number),668,85,54,18,753,72
com verify_number,666,87,52,20,753,72


#### Conclusão sobre o teste com verify_number

O número de falhas aumentou porém também vemos um aumento na identificação de spam.
Assim como o teste com *stem* vimos que o número de mensagens classificados erroneamente como spam cresceu.

### Teste com stem = True, verify_number = True e min_count = 1

In [179]:
stem_and_verify_number_result = execute(NaiveBayesClassifier(stem = True, verify_number = True))

spammiest_words ['rep', 'clearanc', 'per', 'assist', 'account', 'attn', 'zzzz', 'systemwork', 'money', 'adv']
hammiest_words ['spambay', '2', 'razor', 'zzzzteana', 'sadev', 'ouch', 'sell', 'apt', 'wed', 'bliss']


In [180]:
stem_and_verify_number_result

Counter({(False, False): 656,
         (True, True): 92,
         (True, False): 47,
         (False, True): 30})

In [181]:
import pandas as pd

df_stem_and_verify_number_test = pd.DataFrame([dict(original_result),
                                      dict(stem_and_verify_number_result)],
                                      index = ['original', 
                                      'com stem e verify_number'])

stem_and_verify_number_test_success = df_stem_and_verify_number_test[(False, False)] + \
                                      df_stem_and_verify_number_test[(True, True)]
stem_and_verify_number_test_fail = df_stem_and_verify_number_test[(False, True)] + \
                                   df_stem_and_verify_number_test[(True, False)]

df_stem_and_verify_number_test['Total Success']= stem_and_verify_number_test_success
df_stem_and_verify_number_test['Total Fail']= stem_and_verify_number_test_fail
df_stem_and_verify_number_test

,"(False, False)","(True, True)","(True, False)","(False, True)",Total Success,Total Fail
original,668,85,54,18,753,72
com stem e verify_number,656,92,47,30,748,77


#### Conclusão sobre o teste com stem e verify_number

Assim como nos testes anteriores vimos o aumento no número de falhas, assim como também um aumento na identificação de spam.


## Conclusão final

In [182]:
import pandas as pd

df_resultado_final = pd.DataFrame([dict(original_result),
                      dict(min_count_2_result),
                      dict(min_count_3_result),
                      dict(min_count_4_result),
                      dict(min_count_5_result),
                      dict(stem_result),
                      dict(verify_number_result),
                      dict(stem_and_verify_number_result)],
                      index = ['original (min_count = 1)', 
                               'min_count = 2', 
                               'min_count = 3', 
                               'min_count = 4', 
                               'min_count = 5',
                               'com stem',
                               'com verify_number',
                               'com stem e verify_number'
                              ])


final_success = df_resultado_final[(False, False)] + df_resultado_final[(True, True)]
final_fail = df_resultado_final[(False, True)] + df_resultado_final[(True, False)]

df_resultado_final['Total Success']= final_success
df_resultado_final['Total Fail']= final_fail
df_resultado_final

,"(False, False)","(True, True)","(True, False)","(False, True)",Total Success,Total Fail
original (min_count = 1),668,85,54,18,753,72
min_count = 2,631,106,33,55,737,88
min_count = 3,614,111,28,72,725,100
min_count = 4,604,107,32,82,711,114
min_count = 5,587,101,38,99,688,137
com stem,656,91,48,30,747,78
com verify_number,666,87,52,20,753,72
com stem e verify_number,656,92,47,30,748,77


- Como resultado final identificamos que nenhum dos parâmetros criados se mostrou, no geral, mais eficaz que a implementação original.

- Também percebemos que, ao olhar exclusivamente para a identificação de spam sem analisar as falhas, a execução com os parâmetros se mostrou mais eficaz em todos os cenários.

## Referências

McKinney, Wes - Python para Análise de Dados: Tratamento de Dados com Pandas, Numpy e IPython, Editora Novatec, 1a Edição, 2019;